# Daily Knowledge Bot Tutorial

## What You'll Learn

- How to authenticate with the Perplexity API
- How to structure API requests and handle responses
- How to implement a simple topic rotation system
- How to save results to files


## Setting Up Your Environment




In [1]:
import requests
import json
import os
from datetime import datetime



from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

API_KEY  = os.getenv('OPENAI_API_KEY')

## Creating the API Request Function

Next, let's create a function that calls the Perplexity API to get an interesting fact about a given topic.



In [19]:
def get_daily_fact(topic):
    """
    Fetches an interesting fact about the given topic using Perplexity API.
    
    Args:
        topic (str): The topic to get a fact about
        
    Returns:
        str: An interesting fact about the topic
    """
    # url = "https://api.perplexity.ai/chat/completions"
    url = "http://localhost:11434/api/chat"
    
    headers = {
        # "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "llama3.2",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that provides interesting, accurate, and concise facts. Respond with only one fascinating fact, kept under 100 words.Please always answer in Chinese"
            },
            {
                "role": "user",
                "content": f"Tell me an interesting fact about {topic} that most people don't know."
            }
        ],
        "max_tokens": 150,
        "temperature": 0.7,
        "stream": False,
    }
    
    try:
        # print(url, headers, data)
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for 4XX/5XX responses
        result = response.json()
        # print(result)
        # return result["choices"][0]["message"]["content"]
        return result["message"]["content"]
    except requests.exceptions.RequestException as e:
        return f"Error making API request: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

Let's test the function with a sample topic

In [20]:
# Try getting a fact about astronomy
test_fact = get_daily_fact("astronomy")
print(test_fact)

太空中的“星系”并非都是星星组成的天体，而是包含数十至数千个星星、行星和其他物质的巨大系统。例如，双子星座中的“M44”是由600多颗小星团组成的一大的星系。


## Creating a File Saving Function

Now, let's create a function to save our fact to a text file so we can keep a record of all the facts we've learned.



In [21]:

def save_fact_to_file(topic, fact):
    """
    Saves the fact to a text file with timestamp.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
        
    Returns:
        str: Path to the saved file
    """
    timestamp = datetime.now().strftime("%Y-%m-%d")
    filename = f"daily_fact_{timestamp}.txt"
    
    with open(filename, "w") as f:
        f.write(f"DAILY FACT - {timestamp}\n")
        f.write(f"Topic: {topic}\n\n")
        f.write(fact)
    
    print(f"Fact saved to {filename}")
    return filename

Test saving a fact to a file

In [24]:

# Test saving our astronomy fact
saved_file = save_fact_to_file("astronomy", test_fact)

# Let's read the file to verify it worked
with open(saved_file, 'r') as f:
    print(f.read())

Fact saved to daily_fact_2025-03-24.txt
DAILY FACT - 2025-03-24
Topic: astronomy

太空中的“星系”并非都是星星组成的天体，而是包含数十至数千个星星、行星和其他物质的巨大系统。例如，双子星座中的“M44”是由600多颗小星团组成的一大的星系。


## Creating the Topic Rotation Function

We want our bot to provide facts on different topics. Let's create a function that selects a topic based on the current day of the month.



In [25]:
def get_todays_topic():
    """
    Selects a topic based on the current day of the month.
    
    Returns:
        str: Today's selected topic
    """
    topics = [
        "astronomy", 
        "history", 
        "biology", 
        "technology", 
        "psychology",
        "ocean life",
        "ancient civilizations",
        "quantum physics",
        "art history",
        "culinary science",
        "linguistics",
        "architecture",
        "mythology",
        "mathematics",
        "geography",
        "music theory",
        "cryptocurrency",
        "neuroscience",
        "climate science",
        "space exploration",
        "anthropology",
        "philosophy",
        "artificial intelligence",
        "genetics",
        "economics",
        "literature",
        "chemistry",
        "geology",
        "zoology",
        "sustainable energy",
        "robotics"
    ]
    
    # Get the current day of the month (1-31)
    day = datetime.now().day
    
    # Use modulo to wrap around if we have more days than topics
    topic_index = (day % len(topics)) - 1
    if topic_index < 0:  # Handle the case where day % len(topics) = 0
        topic_index = 0
    
    today_topic = topics[topic_index]
    print(f"Today's topic is: {today_topic}")
    return today_topic

# Let's test it
today_topic = get_todays_topic()

Today's topic is: genetics


## Putting It All Together

Now, let's create our main function that combines all of these components.



In [26]:

def run_daily_knowledge_bot():
    """
    Main function that runs the daily knowledge bot.
    Gets a fact about today's topic and saves it to a file.
    """
    # Get today's topic
    today_topic = get_todays_topic()
    print(f"Getting today's fact about: {today_topic}")
    
    # Get the fact
    fact = get_daily_fact(today_topic)
    print(f"\nToday's {today_topic} fact: {fact}")
    
    # Save the fact to a file
    saved_file = save_fact_to_file(today_topic, fact)
    
    return {
        "topic": today_topic,
        "fact": fact,
        "file": saved_file,
        "date": datetime.now().strftime("%Y-%m-%d")
    }

## Run the bot!

In [27]:
# Run our daily knowledge bot
result = run_daily_knowledge_bot()

# Display the result in a more structured way
print("\n" + "-"*50)
print("DAILY KNOWLEDGE BOT RESULT")
print("-"*50)
print(f"Date: {result['date']}")
print(f"Topic: {result['topic']}")
print(f"Fact saved to: {result['file']}")
print("-"*50)

Today's topic is: genetics
Getting today's fact about: genetics

Today's genetics fact: 人类基因组中有约5%的基因被称为"非编码基因"，这意味着它们不编码任何蛋白质序列。这些基因通常参与调控基因表达、RNA加工和其他遗传过程，但其功能仍然不完全理解。例如，具有最多非编码基因的物种是细菌，已发现超过3000个非编码基因。
Fact saved to daily_fact_2025-03-24.txt

--------------------------------------------------
DAILY KNOWLEDGE BOT RESULT
--------------------------------------------------
Date: 2025-03-24
Topic: genetics
Fact saved to: daily_fact_2025-03-24.txt
--------------------------------------------------


## Advanced: Other Ways to Share the Daily Fact

Saving to a file is just one way to store our facts. Here are some additional methods we could implement:



### Email the daily fact

Here's how you could send the fact via email using the smtplib library:



In [29]:
def send_email(topic, fact, recipient):
    """
    Sends the daily fact via email.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
        recipient (str): Email address to send to
        
    Note: This is just a template. You would need to fill in your SMTP server details.
    """
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    
    # Email details
    sender_email = "804345178@qq.com"  # Update this
    password = "your_password"  # Update this (consider using app passwords for security)
    
    # Create message
    message = MIMEMultipart()
    message["Subject"] = f"Daily Knowledge: {topic.capitalize()}"
    message["From"] = sender_email
    message["To"] = recipient
    
    # Email body
    current_date = datetime.now().strftime("%Y-%m-%d")
    email_content = f"""
    
    
        Your Daily Interesting Fact - {current_date}
        Topic: {topic.capitalize()}
        {fact}
        
        Brought to you by Daily Knowledge Bot
    
    
    """
    
    message.attach(MIMEText(email_content, "html"))
    
    # Connect to server and send (commented out as this requires actual credentials)
    """
    with smtplib.SMTP_SSL("smtp.example.com", 465) as server:  # Update with your SMTP server
        server.login(sender_email, password)
        server.send_message(message)
    print(f"Email sent to {recipient}")
    """
    
    # Since we're just displaying the concept, print what would be sent
    print(f"\nWould send email to: {recipient}")
    print(f"Subject: Daily Knowledge: {topic.capitalize()}")
    print("Content:")
    print(f"Topic: {topic.capitalize()}")
    print(f"Fact: {fact}")

## Save to a database

If you wanted to build up a collection of facts over time, saving to a database would be ideal. Here's a simple example using SQLite:



In [30]:
def save_to_database(topic, fact):
    """
    Saves the fact to an SQLite database.
    
    Args:
        topic (str): The topic of the fact
        fact (str): The fact content
    """
    import sqlite3
    from datetime import datetime
    
    # Connect to database (creates it if it doesn't exist)
    conn = sqlite3.connect("daily_facts.db")
    cursor = conn.cursor()
    
    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS facts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        topic TEXT,
        fact TEXT
    )
    """)
    
    # Insert the new fact
    current_date = datetime.now().strftime("%Y-%m-%d")
    cursor.execute(
        "INSERT INTO facts (date, topic, fact) VALUES (?, ?, ?)",
        (current_date, topic, fact)
    )
    
    # Commit and close
    conn.commit()
    conn.close()
    
    print(f"Fact saved to database with topic '{topic}'")

测试数据库

In [31]:
# Run our daily knowledge bot
result = run_daily_knowledge_bot()

save_to_database(result['topic'], result['fact'])


Today's topic is: genetics
Getting today's fact about: genetics

Today's genetics fact: 遗传学中的一个有趣事实是：人体的基因组中，约60%的DNA分子并不是编码蛋白质序列，而是编码"非编码"RNA。这些RNA被称为ncRNA（非编码核糖体RNA），而它们在许多生物过程中起着至关重要的作用，例如调控基因表达、修复DNA损伤等。ncRNA的存在使我们了解到遗传学中的更广泛和复杂的机制。
Fact saved to daily_fact_2025-03-24.txt
Fact saved to database with topic 'genetics'
